# Segmenting and Clustering Neighborhoods in Toronto

Toronto information taken from:
    https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Get table from wikipedia and assign NaN to Not assigned

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url).text

soup = BeautifulSoup(page,"lxml")
#print(soup.prettify())

table = soup.find('table',{'class':'wikitable sortable'})
T_neigh = pd.read_html(str(table), header=0, na_values="Not assigned")[0]
print(T_neigh.shape)
T_neigh.head(10)

(289, 3)


,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
9,M8A,NaN,NaN


### Cells with a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
(see index 8)

In [3]:
df = pd.DataFrame(columns= T_neigh.columns).reindex_like(T_neigh)
for i, row in T_neigh.iterrows():
    if ((row[1] is not np.NaN) and (row[2] is np.NaN)):
        row[2] = row[1]
        df.iloc[i,:] = list(row)
        
    else:
        df.iloc[i,:] = list(row)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,NaN,NaN


### Eliminating Not assigned Postal Codes

In [4]:
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

### Grouping by Postal code and Bourough

In [5]:
df_tor = df.groupby(["Postcode", "Borough"])['Neighbourhood'].apply(', '.join)
df0 = pd.DataFrame(df_tor)
tor = df0.reset_index()
tor.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
tor.shape

(103, 3)

In [ ]:
!pip install geocoder 

### Assigning Coordinates

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]